In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

In [3]:
# Carregar dados
data = pd.read_csv('data_/TSLA.csv')
data['Date'] = pd.to_datetime(data['Date'])

In [4]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-01-09,22.366667,22.900000,22.098000,22.568666,22.568666,81493500
1,2019-01-10,22.293333,23.025999,22.119333,22.997999,22.997999,90846000
2,2019-01-11,22.806000,23.227333,22.584667,23.150667,23.150667,75586500
3,2019-01-14,22.825333,22.833332,22.266666,22.293333,22.293333,78709500
4,2019-01-15,22.333332,23.253332,22.299999,22.962000,22.962000,90849000


In [5]:
# Definir rótulos para classificação
data['Return'] = data['Close'].pct_change().shift(-1)
data.dropna(inplace=True)

In [6]:
def label_return(row):
    if row['Return'] > 0.01:
        return 0
    elif row['Return'] < -0.01:
        return 1
    else:
        return 2

data['Label'] = data.apply(label_return, axis=1)

In [7]:
data['Label']

0       0
1       2
2       1
3       0
4       2
       ..
1106    0
1107    0
1108    0
1109    0
1110    0
Name: Label, Length: 1111, dtype: int64

In [8]:
# Vamos utilizar o fechamento, máxima e mínima das ações
input_data = data[['Close', 'High', 'Low']].values

# Normalizar dados
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(input_data)

In [9]:
# Função para criar sequências de dados
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data['Label'].iloc[i + seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [10]:
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Return,Label
0,2019-01-09,22.366667,22.900000,22.098000,22.568666,22.568666,81493500,0.019023,0
1,2019-01-10,22.293333,23.025999,22.119333,22.997999,22.997999,90846000,0.006638,2
2,2019-01-11,22.806000,23.227333,22.584667,23.150667,23.150667,75586500,-0.037033,1
3,2019-01-14,22.825333,22.833332,22.266666,22.293333,22.293333,78709500,0.029994,0
4,2019-01-15,22.333332,23.253332,22.299999,22.962000,22.962000,90849000,0.004703,2


In [11]:
data.reset_index()

,index,Date,Open,High,Low,Close,Adj Close,Volume,Return,Label
0,0,2019-01-09,22.366667,22.900000,22.098000,22.568666,22.568666,81493500,0.019023,0
1,1,2019-01-10,22.293333,23.025999,22.119333,22.997999,22.997999,90846000,0.006638,2
2,2,2019-01-11,22.806000,23.227333,22.584667,23.150667,23.150667,75586500,-0.037033,1
3,3,2019-01-14,22.825333,22.833332,22.266666,22.293333,22.293333,78709500,0.029994,0
4,4,2019-01-15,22.333332,23.253332,22.299999,22.962000,22.962000,90849000,0.004703,2
...,...,...,...,...,...,...,...,...,...,...
1106,1106,2023-06-01,202.589996,209.800003,199.369995,207.520004,207.520004,148029900,0.031081,0
1107,1107,2023-06-02,210.149994,217.250000,209.750000,213.970001,213.970001,164129000,0.017012,0
1108,1108,2023-06-05,217.800003,221.289993,214.520004,217.610001,217.610001,151143100,0.017003,0
1109,1109,2023-06-06,216.139999,221.910004,212.529999,221.309998,221.309998,146911600,0.014731,0


In [13]:
data.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'Return',
       'Label'],
      dtype='object')

In [15]:
data = data.drop('Date', axis=1)

In [16]:
data

,Open,High,Low,Close,Adj Close,Volume,Return,Label
0,22.366667,22.900000,22.098000,22.568666,22.568666,81493500,0.019023,0
1,22.293333,23.025999,22.119333,22.997999,22.997999,90846000,0.006638,2
2,22.806000,23.227333,22.584667,23.150667,23.150667,75586500,-0.037033,1
3,22.825333,22.833332,22.266666,22.293333,22.293333,78709500,0.029994,0
4,22.333332,23.253332,22.299999,22.962000,22.962000,90849000,0.004703,2
...,...,...,...,...,...,...,...,...
1106,202.589996,209.800003,199.369995,207.520004,207.520004,148029900,0.031081,0
1107,210.149994,217.250000,209.750000,213.970001,213.970001,164129000,0.017012,0
1108,217.800003,221.289993,214.520004,217.610001,217.610001,151143100,0.017003,0
1109,216.139999,221.910004,212.529999,221.309998,221.309998,146911600,0.014731,0


In [17]:
SEQ_LENGTH = 10

X, y = create_sequences(data, SEQ_LENGTH)

In [18]:
# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

In [19]:
# Construir o modelo RNN
model = Sequential()
model.add(SimpleRNN(50, activation='relu', 
                    input_shape=(X_train.shape[1], 
                                 X_train.shape[2])))

model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

In [20]:
print(X_train)

[[[ 2.23666668e+01  2.28999996e+01  2.20979996e+01 ...  8.14935000e+07
    1.90233984e-02  0.00000000e+00]
  [ 2.22933331e+01  2.30259991e+01  2.21193333e+01 ...  9.08460000e+07
    6.63831658e-03  2.00000000e+00]
  [ 2.28059998e+01  2.32273331e+01  2.25846672e+01 ...  7.55865000e+07
   -3.70328047e-02  1.00000000e+00]
  ...
  [ 2.15333328e+01  2.18086662e+01  1.99820004e+01 ...  3.62262000e+08
   -1.10501400e-02  1.00000000e+00]
  [ 2.03213329e+01  2.05333328e+01  1.97000008e+01 ...  1.81000500e+08
   -3.79030995e-02  1.00000000e+00]
  [ 1.95000000e+01  1.96333332e+01  1.87793331e+01 ...  1.87950000e+08
    1.36305226e-02  0.00000000e+00]]

 [[ 2.22933331e+01  2.30259991e+01  2.21193333e+01 ...  9.08460000e+07
    6.63831658e-03  2.00000000e+00]
  [ 2.28059998e+01  2.32273331e+01  2.25846672e+01 ...  7.55865000e+07
   -3.70328047e-02  1.00000000e+00]
  [ 2.28253326e+01  2.28333321e+01  2.22666664e+01 ...  7.87095000e+07
    2.99940273e-02  0.00000000e+00]
  ...
  [ 2.03213329e+01  2.0

In [21]:
# Treinar o modelo
model.fit(X_train, y_train, 
          epochs=100, batch_size=64, 
          validation_data=(X_test, y_test))

Epoch 1/100
14/14 [==============================] - 2s 34ms/step - loss: 59151628.0000 - accuracy: 0.3045 - val_loss: 16732180.0000 - val_accuracy: 0.3710
Epoch 2/100
14/14 [==============================] - 0s 12ms/step - loss: 12571349.0000 - accuracy: 0.3443 - val_loss: 7530064.0000 - val_accuracy: 0.2579
Epoch 3/100
14/14 [==============================] - 0s 10ms/step - loss: 6533622.5000 - accuracy: 0.3432 - val_loss: 2467632.7500 - val_accuracy: 0.3439
Epoch 4/100
14/14 [==============================] - 0s 8ms/step - loss: 3486517.0000 - accuracy: 0.3477 - val_loss: 1081900.5000 - val_accuracy: 0.4118
Epoch 5/100
14/14 [==============================] - 0s 7ms/step - loss: 2061074.1250 - accuracy: 0.3602 - val_loss: 654380.7500 - val_accuracy: 0.3575
Epoch 6/100
14/14 [==============================] - 0s 8ms/step - loss: 1429519.8750 - accuracy: 0.3636 - val_loss: 325487.5625 - val_accuracy: 0.3846
Epoch 7/100
14/14 [==============================] - 0s 11ms/step - loss: 1035

In [22]:
# Avaliar o modelo
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy:.4f}")

7/7 [==============================] - 0s 2ms/step
Accuracy: 0.3756
